In [ ]:
import cv2
print(cv2.__version__)                     #Check for latest version

from tensorflow.keras.preprocessing.image import ImageDataGenerator       
from tensorflow.keras.preprocessing import image
from tensorflow.keras.optimizers import RMSprop
import matplotlib.pyplot as plt
import tensorflow as tf
import numpy as np
import os

# Load and display an image from the Dataset
img = image.load_img('Documents/Dataset/Happy or Sad AJAY/training/happy_ajay/1.png')
plt.imshow(img)

# Check image shape
cv2.imread('Documents/Dataset/Happy or Sad AJAY/training/happy_ajay/1.png').shape

# Data Generators for Training and Validation
train = ImageDataGenerator(rescale = 1/255)            #Normalization 
validation = ImageDataGenerator(rescale = 1/255)

train_dataset = train.flow_from_directory('Documents/Dataset/Happy or Sad AJAY/training',
                                          target_size = (200,200), 
                                          batch_size = 3, 
                                          class_mode = 'binary')

validation_dataset = validation.flow_from_directory('Documents/Dataset/Happy or Sad AJAY/Validation',
                                                    target_size = (200,200), 
                                                    batch_size = 3, 
                                                    class_mode = 'binary')

# Check class indices (2 Classes Happy or Sad)
train_dataset.class_indices

# Define the CNN Model
model = tf.keras.models.Sequential([
    # First Convolutional Layer
    tf.keras.layers.Conv2D(16, (3, 3), activation='relu', input_shape=(200, 200, 3)),
    tf.keras.layers.MaxPool2D(2, 2),

    # Second Convolutional Layer
    tf.keras.layers.Conv2D(32, (3, 3), activation='relu'),
    tf.keras.layers.MaxPool2D(2, 2),

    # Third Convolutional Layer
    tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
    tf.keras.layers.MaxPool2D(2, 2),

    # Flattening Layer
    tf.keras.layers.Flatten(),

    # Fully Connected Layer
    tf.keras.layers.Dense(512, activation='relu'),

    # Output Layer
    tf.keras.layers.Dense(1, activation='sigmoid')
])

# Compile the Model
model.compile(loss = 'binary_crossentropy',
              optimizer = RMSprop(lr=0.001),
              metrics = ['accuracy'])

# Train the Model using fit
model_fit = model.fit(
    train_dataset,
    steps_per_epoch=3,
    epochs=50,
    validation_data=validation_dataset
)

# Testing the Model on New Unseen Data
dir_path = 'Documents/Dataset/Happy or Sad AJAY/testing'

for i in os.listdir(dir_path):
    img = image.load_img(os.path.join(dir_path, i), target_size=(200, 200))  # Load image
    plt.imshow(np.array(img))  # Display image
    plt.show()

    X = image.img_to_array(img)
    X = np.expand_dims(X, axis=0)  # Expand dimensions for prediction
    images = np.vstack([X])  # Stack the image for prediction

    # Predict using the model
    val = model.predict(images)

    # Output based on prediction result
    if val[0] == 0:
        print('Ajay is Happy')
    else:
        print('Ajay is Sad')
